In [1]:
import openai
openai.api_key = 'Your key'
ablation = 'random'#['ALKERec','RAD-BC','random','easiest','hardest']

In [2]:
import argparse
import numpy as np
from utils.dataset import Dataset,DataIterator,get_DataLoader_few_shot

def parse_args(name):   
    parser = argparse.ArgumentParser(description="Run .")  
    parser.add_argument('--dataset', nargs='?', default=name,
                        help='Choose a dataset.')
    parser.add_argument('--hidden_factor', type=int, default=10,
                        help='Number of hidden factors.')
    parser.add_argument('--lr', type=float, default=0.001,
                        help='Learning rate.')    
    return parser.parse_args(args=[])

In [3]:
args = parse_args('Games')
data = Dataset(args)
    

loading data: meta_data

loading data: interaction_data

split data



In [4]:
class Database_LLM(object):
    def __init__(self,path):
        #|session_id|rec_result|hint_position|
        self.dir = path
        self.exist_sessions()
    def exist_sessions(self):
        # 计算文件的行数
        self.exist_session = set()
        with open(self.dir, 'r', encoding='utf-8') as file:
            for line in file:
                session_id,rec_result,hint_position =line.split('\t')
                self.exist_session.add(int(session_id))
    def write(self,session_id,rec_result,hint_position):
        with open(self.dir,'a') as f:
            f.write("%d\t%s\t%d\n"%(session_id,str(rec_result),hint_position))
        self.exist_sessions()
    def get_all_dict(self):
        table_rec = dict()
        table_hit = dict()
        with open(self.dir, 'r', encoding='utf-8') as file:
            for line in file:
                session_id,rec_result,hint_position =line.split('\t')
                table_rec[int(session_id)] = eval(rec_result)
                table_hit[int(session_id)] = eval(hint_position[:-1])
        return table_rec,table_hit
    def write_sub_database(self,sub_dir,session_id):
        with open(sub_dir,'a') as f:
            f.write("%d\n"%session_id)
    def read_sub_database(self,sub_dir):
        r = set()
        with open(sub_dir) as f:
            for line in f:
                r.add(int(line[:-1]))
        return r

In [5]:
path = './datasets/%s/%s_LLM_Database.data'%(args.dataset,args.dataset)
# path_initial = './datasets/%s_LLM_initial_session.data'%args.dataset
database = Database_LLM(path)

In [6]:
import pandas as pd
df = pd.read_pickle('./datasets/%s/%s_prediction.pkl'%(args.dataset,args.dataset))

In [7]:
#确保target item在candidate set中
A = 'target'
B = 'student_top_item_'
C = 'teacher_top_item'
df[B] = df.apply(lambda row: np.array(row[B][:-1].tolist() + [row[A]]) if row[A] not in row[B] else row[B], axis=1)
df[C] = df.apply(lambda row: np.array(row[C][:-1].tolist() + [row[A]]) if row[A] not in row[C] else row[C], axis=1)
# args.batch_size

In [8]:
#constrcut sub_df where sub_df 
sub_df = df[df['session_id'].isin(database.exist_session)]
table_rec,table_hit = database.get_all_dict()
sub_df['LLM_pred'] = [table_rec[session_id] for session_id in sub_df.session_id]
sub_df['LLM_hit'] = [table_hit[session_id] for session_id in sub_df.session_id]
print(len(sub_df))



5348


/tmp/ipykernel_3731621/2179324063.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['LLM_pred'] = [table_rec[session_id] for session_id in sub_df.session_id]
/tmp/ipykernel_3731621/2179324063.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['LLM_hit'] = [table_hit[session_id] for session_id in sub_df.session_id]


In [9]:

#ChatGPT zero shot performance
import os
import time
import numpy as np
import re


class Prompt_class(object):
    def __init__(self,train_df,data):
        self.train_df = train_df
        self.data = data
    def ids2names(self,ids):
        r = '{'
        for item in ids:
            r = r + '<ID_%d:'%item+str(self.data.id2title[item])+'>, '
        r = r[:-2] + '}'
        return r
    def generate_prompt(self,idx):
        df_idx = self.train_df[self.train_df['session_id']==idx].iloc[0]
        candiate = set(df_idx['teacher_top_item'])
        if df_idx['target'] not in candiate:
            candiate.pop()
            candiate.add(df_idx['target'])
        feature = set(df_idx['feature'] )
        p1 = 'You are an AI recommender system, please make accurate recommendation to the user according to his/her behaviors. '
        h1 = 'There are some hints for recommendation:' #Notice user may prefer the similar and same series items. Do not focus on diversity.'
        h11 = "Platform and Accessory Compatibility: The system heavily factors in the user's current gaming platform, consistently recommending games and accessories that are compatible with it."
        h12 = "Series and Genre Loyalty: Users are frequently recommended titles from game series they have previously engaged with, as well as games within the same or similar genres."
        h13 = "Special Editions and Bundles: The system often suggests special or collector's editions of games and bundled offers, thereby potentially increasing their investment and engagement."
        h14 = "In disregard of diversity in game genres or series. For example, Although the user already owns the game, there would still be interest in same game, e.g., its the special version."
        p2 = 'The users has bought %d video games, whose titles are namely %s.'%(len(feature),self.ids2names(feature))
        p3 = 'Based on these interactions, recommend a ranking list of items for the user from a candidate set: {%s}. '%self.ids2names(candiate)
        p4 = 'Specifically,  ranking ALL these candidate items and provide EXACTLY 25 items for recommendations, where item in the top position indicates the higher recommendation intent.'
        p5 = 'Just output WITHOUT ANY OHTER MESSAGE: [<ID1>,...,<ID25>], where with <ID?> surrounding JUST the ID WITHOUT title and split all IDs with comma.'
        Prompt = p1 + h1 + h11 +h12 + h13 + h14 + p2 + p3 + p4 + p5
        # print(Prompt)
        # print('\n\n\n')
        # print('Groud Truth:%s'%self.ids2names([df_idx['target']]))
        return Prompt,df_idx['target']
    def get_completion(self, prompt, model="gpt-4"):
        messages = [{"role": "assistant", "content": prompt}]
        response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
        )
        return response.choices[0].message["content"]
    def response2list(self,response):
        return [int(ids) for ids in re.findall(r'ID_(\d+)', response)]

In [10]:
promp_generator = Prompt_class(df,data)

In [11]:
def MAXMIN_vanilla(N,k=0.6,distribution='exp'):
    if distribution == 'exp':
        alpha = 10
        g = np.array([float(i) for i in range(1,N+1)])**(-alpha)
    l = 0.5 * g
    gk = g/(g+l)
    G=np.cumsum(gk)
    hk = 1/(g+l)
    H=np.cumsum(hk)
    k_max = (G + len(g)*(k-1))/H
    k_star = np.argmax(k_max)
    p = 1/(g+l)
    p[k_star:] = 0
    p = p / np.sum(p)
    return p,k_star

In [12]:
def MAXMIN(N,k1=0.1,k2=0.5,distribution='exp',g=None):
    if distribution == 'exp':
        alpha = 10
        g = np.array([float(i) for i in range(1,N+1)])**(-alpha)
    if distribution == 'self-define': 
        g = g
    l = 0.5 * g
    f = 0.5 * g
    gk = f/(f+l)
    gk[int(N*(1-k1)):] = (g/(g+l))[int(N*(1-k1)):] 
    G=np.cumsum(gk)
    hk = 1/(f+l)
    hk[int(N*(1-k1)):] = (1/(g+l))[int(N*(1-k1)):] 
    H=np.cumsum(hk)
    k_max = (G + len(g)*(k1+k2-1))/H
    k_star = np.argmax(k_max)
    p = 1/(f+l)
    p[int((1-k1)*N):k_star] = (1/(g+l))[int((1-k1)*N):k_star]
    p[k_star:] = 0
    p = p / np.sum(p)
    return p,k_star

In [13]:
def active_learn_ours(sub_df):
    #从50个random出来的sample里面选择10个in-distribution的sample；
    N = len(sub_df)
    probabilities,k_star = MAXMIN(N)
    print([N,k_star])
    elements = np.arange(N)
    sample_index = np.random.choice(elements, size=1, p=probabilities,replace=False)[0]
    select_line = sub_df.iloc[sample_index]
    sub_df = sub_df.drop(sub_df.index[sample_index])
    return sub_df, select_line
def active_learn_vanilla(sub_df):
    #从50个random出来的sample里面选择10个in-distribution的sample；
    N = len(sub_df)
    probabilities,k_star = MAXMIN_vanilla(N)
    print([N,k_star])
    elements = np.arange(N)
    sample_index = np.random.choice(elements, size=1, p=probabilities,replace=False)[0]
    select_line = sub_df.iloc[sample_index]
    sub_df = sub_df.drop(sub_df.index[sample_index])
    return sub_df, select_line
def active_learn_random(sub_df):
    #从50个random出来的sample里面选择10个in-distribution的sample；
    select_line = sub_df.sample(1)
    index = select_line.index
    select_line = select_line.iloc[0]
    sub_df = sub_df.drop(index)
    return sub_df, select_line
def active_learn_easiest(sub_df):
    #
    sampled_df = sub_df
    max_index = sampled_df['difference'].idxmax()
    select_line = sampled_df.loc[max_index]
    sub_df = sub_df.drop(max_index)
    return sub_df, select_line
def active_learn_hardest(sub_df):
    #
    sampled_df = sub_df
    max_index = sampled_df['difference'].idxmin()
    select_line = sampled_df.loc[max_index]
    sub_df = sub_df.drop(max_index)
    return sub_df, select_line

In [14]:
import time

# 开始计时
start_time = time.time()




if 'ALKERec' in ablation:
    select_method = active_learn_ours
    df = df.sort_values('difference')
if 'RAD-BC' in ablation:
    select_method = active_learn_vanilla
    df = df.sort_values('difference')
if ablation == 'random':
    select_method = active_learn_random
if ablation == 'easiest':
    select_method = active_learn_easiest
if ablation == 'hardest':
    select_method = active_learn_hardest
# ablation_sessions = set()
ablation_sessions = database.read_sub_database('./datasets/%s/%s_%s.base'%(args.dataset,args.dataset,ablation))

df = df[~df['session_id'].isin(ablation_sessions)]

n = 0
for i in range(20):
    df, select_line = select_method(df)
    session_idx = select_line.session_id
    if session_idx in database.exist_session:
        #如果已经预测过了，只需记录一下就可以；
        database.write_sub_database('./datasets/%s/%s_%s.base'%(args.dataset,args.dataset,ablation),session_idx)
        print("existing")
    else:
        n += 1
        #如果没预测过
        #构建prompt
        p,gt = promp_generator.generate_prompt(session_idx)
        #基于ChatGPT生成结果
        response = promp_generator.get_completion(p,model='gpt-4-turbo-2024-04-09')
        print(response)
        #转化成top5推荐
        top_results = promp_generator.response2list(response)
        #输出结果：
        try:
            position = top_results.index(gt)
        except:
            position = 25
            
        print('session:%d,teacher_position:%d,student_position:%d,HIT:%d'%(session_idx,select_line.teacher_position,select_line.student_position,position))
        #写入大库：
        database.write(session_idx,top_results,position)        
        #写入ablation库
        database.write_sub_database('./datasets/%s/%s_%s.base'%(args.dataset,args.dataset,ablation),session_idx)
# 结束计时
end_time = time.time()
# 计算运行时间
elapsed_time = end_time - start_time
print(f"一个instance的运行时间：{elapsed_time/n}秒")


existing
existing
[<ID_23854>,<ID_11854>,<ID_13654>,<ID_8124>,<ID_17440>,<ID_10803>,<ID_19472>,<ID_20995>,<ID_5616>,<ID_15962>,<ID_18098>,<ID_7012>,<ID_10087>,<ID_3056>,<ID_6737>,<ID_10313>,<ID_8243>,<ID_9307>,<ID_20819>,<ID_23346>,<ID_22290>,<ID_2874>,<ID_5249>,<ID_7252>,<ID_7012>]
session:10395,teacher_position:0,student_position:223,HIT:1
existing
[<ID_5424>,<ID_7380>,<ID_24793>,<ID_24913>,<ID_22102>,<ID_18288>,<ID_17265>,<ID_19895>,<ID_23355>,<ID_25893>,<ID_20384>,<ID_23533>,<ID_24053>,<ID_19482>,<ID_24971>,<ID_24198>,<ID_26087>,<ID_24601>,<ID_10612>,<ID_19874>,<ID_21103>,<ID_19043>,<ID_18288>,<ID_22008>,<ID_12666>]
session:1560,teacher_position:0,student_position:12,HIT:0
existing
[<ID_9296>,<ID_21681>,<ID_26044>,<ID_18582>,<ID_11807>,<ID_24844>,<ID_2179>,<ID_22661>,<ID_22660>,<ID_24962>,<ID_16258>,<ID_14464>,<ID_17169>,<ID_14375>,<ID_14243>,<ID_14522>,<ID_13376>,<ID_6315>,<ID_14156>,<ID_21206>,<ID_25027>,<ID_20426>,<ID_20734>,<ID_2037>,<ID_17390>]
session:1395,teacher_position:0,

In [18]:
5.324412692676891*500/60

44.37010577230742

In [19]:
5.324412692676891*20091/60

1782.8795901428568

In [23]:
0.19/n *20091

347.0263636363636